# TASK4 - feature engineering

Propose better solution for feature engineering than the one given by us.

-----------------------------------------------------

### LOAD PACKAGES

In [1]:
import random
import time
import datacompy
import sys
import IPython
from IPython import display
import matplotlib 
import pandas as pd
import numpy as np 
import scipy as sp 
import sklearn
from collections import Counter
import pickle as pkl

In [2]:
import sys  
sys.path.insert(0, '../src/')

In [3]:
import functions
from functions import detect_outliers, missing_ratio, simple_feature_eng, completing, label_encoding, fare_category

/home/dklimarczyk/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

In [5]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

In [7]:
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('dark')
pylab.rcParams['figure.figsize'] = 14,5
seed =40
plt.style.use('fivethirtyeight')

-----------------------------------------------------

## Import of raw data (train) and validation data (val/test)

In [8]:
train = pd.read_csv('../data/train.csv')

In [9]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
train.shape

(891, 12)

In [11]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [12]:
train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
347,348,1,3,"Davison, Mrs. Thomas Henry (Mary E Finck)",female,NaN,1,0,386525,16.1000,NaN,S
212,213,0,3,"Perkin, Mr. John Henry",male,22.0,0,0,A/5 21174,7.2500,NaN,S
550,551,1,1,"Thayer, Mr. John Borland Jr",male,17.0,0,2,17421,110.8833,C70,C
758,759,0,3,"Theobald, Mr. Thomas Leonard",male,34.0,0,0,363294,8.0500,NaN,S
18,19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vande...",female,31.0,1,0,345763,18.0000,NaN,S
380,381,1,1,"Bidois, Miss. Rosalie",female,42.0,0,0,PC 17757,227.5250,NaN,C
743,744,0,3,"McNamee, Mr. Neal",male,24.0,1,0,376566,16.1000,NaN,S
549,550,1,2,"Davies, Master. John Morgan Jr",male,8.0,1,1,C.A. 33112,36.7500,NaN,S
518,519,1,2,"Angle, Mrs. William A (Florence ""Mary"" Agnes H...",female,36.0,1,0,226875,26.0000,NaN,S
243,244,0,3,"Maenpaa, Mr. Matti Alexanteri",male,22.0,0,0,STON/O 2. 3101275,7.1250,NaN,S


-----------------------------------------------------

In [13]:
test  = pd.read_csv('../data/test.csv')

In [14]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
test.shape

(418, 11)

In [16]:
test.shape[0]+train.shape[0]

1309

-----------------------------------------------------

### Comparison of two data sets train and test

In [17]:
compare = compare = datacompy.Compare(train,
                                      test,   
                                      join_columns='Name', # you can also check it on other columns but it doesn't metter
                                      df1_name="train",
                                      df2_name="test")

In [18]:
print(compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0     train       12   891
1      test       11   418

Column Summary
--------------

Number of columns in common: 11
Number of columns in train but not in test: 1
Number of columns in test but not in train: 0

Row Summary
-----------

Matched on: name
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 2
Number of rows in train but not in test: 889
Number of rows in test but not in train: 416

Number of rows with some compared columns unequal: 2
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 6
Total number of values which compare unequal: 9

Columns with Unequal Values or Types
------------------------------------

        Column train dtype test dtype  # Unequal    Max Diff  # Null D

-----------------------------------------------------

In [19]:
print('Number of null values in Train clumns:\n', train.isnull().sum())

Number of null values in Train clumns:
 passengerid      0
survived         0
pclass           0
name             0
sex              0
age            177
sibsp            0
parch            0
ticket           0
fare             0
cabin          687
embarked         2
dtype: int64


In [20]:
print('Number of null values in Validation dataset columns:\n', test.isnull().sum())

Number of null values in Validation dataset columns:
 passengerid      0
pclass           0
name             0
sex              0
age             86
sibsp            0
parch            0
ticket           0
fare             1
cabin          327
embarked         0
dtype: int64


-----------------------------------------------------

#### Outlier detection and df cleaning from them

In [21]:
# outliers from age, sibsp , parch and fare
outliers_to_drop = detect_outliers(train,2,["age","sibsp","parch","fare"])

# outliers rows
train.loc[outliers_to_drop] 

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.00,C23 C25 C27,S
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.00,C23 C25 C27,S
159,160,0,3,"Sage, Master. Thomas Henry",male,NaN,8,2,CA. 2343,69.55,NaN,S
180,181,0,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.55,NaN,S
201,202,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.55,NaN,S
324,325,0,3,"Sage, Mr. George John Jr",male,NaN,8,2,CA. 2343,69.55,NaN,S
341,342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.00,C23 C25 C27,S
792,793,0,3,"Sage, Miss. Stella Anna",female,NaN,8,2,CA. 2343,69.55,NaN,S
846,847,0,3,"Sage, Mr. Douglas Bullen",male,NaN,8,2,CA. 2343,69.55,NaN,S
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.55,NaN,S


In [22]:
train = train.drop(outliers_to_drop, axis = 0).reset_index(drop=True)

In [23]:
train.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Concatenate test and train datasets

In [24]:
all_df = pd.concat([train, test], sort = False).reset_index(drop=True)

In [25]:
all_df.shape

(1299, 12)

In [26]:
print(train.shape)
print(test.shape)
print(all_df.shape[0] == train.shape[0]+test.shape[0])

(881, 12)
(418, 11)
True


#### Find Missing Ratio of Dataset

In [27]:
missing_ratio(all_df)

,"Missing value, %",N unique value,dtype
passengerid,0.0,1299,int64
survived,32.0,2,float64
pclass,0.0,3,int64
name,0.0,1297,object
sex,0.0,2,object
age,20.0,98,float64
sibsp,0.0,7,int64
parch,0.0,8,int64
ticket,0.0,929,object
fare,0.0,281,float64


### Cleaning

In [28]:
completing(all_df)

In [29]:
print(train.isnull().sum())
print("-"*50)
print(test.isnull().sum())

passengerid      0
survived         0
pclass           0
name             0
sex              0
age            170
sibsp            0
parch            0
ticket           0
fare             0
cabin          680
embarked         2
dtype: int64
--------------------------------------------------
passengerid      0
pclass           0
name             0
sex              0
age             86
sibsp            0
parch            0
ticket           0
fare             1
cabin          327
embarked         0
dtype: int64


-----------------------------------------------------

### Actual feature engineering

In [30]:
simple_feature_eng(all_df)

/home/dklimarczyk/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,passengerid,survived,pclass,name,sex,age,sibsp,parch,fare,embarked,...,t_SOTONO2,t_SOTONOQ,t_SP,t_STONO,t_STONO2,t_STONOQ,t_SWPP,t_WC,t_WEP,t_x
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,...,0,0,0,0,0,0,0,0,0,0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,...,0,0,0,0,0,0,0,0,0,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,...,0,0,0,0,1,0,0,0,0,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,...,0,0,0,0,0,0,0,0,0,1
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294,1305,NaN,3,"Spector, Mr. Woolf",male,28.0,0,0,8.0500,S,...,0,0,0,0,0,0,0,0,0,0
1295,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,108.9000,C,...,0,0,0,0,0,0,0,0,0,0
1296,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,7.2500,S,...,0,1,0,0,0,0,0,0,0,0
1297,1308,NaN,3,"Ware, Mr. Frederick",male,28.0,0,0,8.0500,S,...,0,0,0,0,0,0,0,0,0,1


In [31]:
all_df['fare_category'] = all_df['fare'].apply(fare_category) 

In [32]:
all_df['small_f'] = all_df['familysize'].map(lambda x: 1 if  x == 2  else 0)

all_df['med_f'] = all_df['familysize'].map(lambda x: 1 if 3 <= x <= 4 else 0)

all_df['large_f'] = all_df['familysize'].map(lambda x: 1 if x >= 5 else 0)

-----------------------------------------------------

### Clean-up of rare title names

In [33]:
all_df['title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', 'Dona'], dtype=object)

In [34]:
all_df['title'].value_counts()

Mr              753
Miss            255
Mrs             197
Master           60
Dr                8
Rev               8
Col               4
Mlle              2
Ms                2
Major             2
Jonkheer          1
Don               1
Lady              1
Sir               1
Dona              1
Mme               1
the Countess      1
Capt              1
Name: title, dtype: int64

Why 10 as a minimum  -- http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/

In [35]:
a_min = 10
# creatying a true/false series with title name as a index
title_names = (all_df['title'].value_counts() < a_min) 

In [36]:
title_names

Mr              False
Miss            False
Mrs             False
Master          False
Dr               True
Rev              True
Col              True
Mlle             True
Ms               True
Major            True
Jonkheer         True
Don              True
Lady             True
Sir              True
Dona             True
Mme              True
the Countess     True
Capt             True
Name: title, dtype: bool

In [37]:
# Every title name with True vale will be applied as "Misc" for "Miscellaneous"
all_df['title'] = all_df['title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)

In [38]:
print(all_df['title'].value_counts())

Mr        753
Miss      255
Mrs       197
Master     60
Misc       34
Name: title, dtype: int64


In [39]:
all_df.info()
all_df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   passengerid    1299 non-null   int64  
 1   survived       881 non-null    float64
 2   pclass         1299 non-null   int64  
 3   name           1299 non-null   object 
 4   sex            1299 non-null   object 
 5   age            1299 non-null   float64
 6   sibsp          1299 non-null   int64  
 7   parch          1299 non-null   int64  
 8   ticket         1299 non-null   object 
 9   fare           1299 non-null   float64
 10  embarked       1299 non-null   object 
 11  familysize     1299 non-null   int64  
 12  isalone        1299 non-null   int64  
 13  no_fare        1299 non-null   int64  
 14  title          1299 non-null   object 
 15  fare_category  1299 non-null   int64  
 16  small_f        1299 non-null   int64  
 17  med_f          1299 non-null   int64  
 18  large_f 

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,embarked,familysize,isalone,no_fare,title,fare_category,small_f,med_f,large_f
1057,1068,NaN,2,"Sincock, Miss. Maude",female,20.00,0,0,CA,36.7500,S,1,1,0,Miss,4,0,0,0
281,287,1.0,3,"de Mulder, Mr. Theodore",male,30.00,0,0,x,9.5000,S,1,1,0,Mr,2,0,0,0
454,462,0.0,3,"Morley, Mr. William",male,34.00,0,0,x,8.0500,S,1,1,0,Mr,2,0,0,0
77,79,1.0,2,"Caldwell, Master. Alden Gates",male,0.83,0,2,x,29.0000,S,3,0,0,Master,3,0,1,0
337,345,0.0,2,"Fox, Mr. Stanley Hubert",male,36.00,0,0,x,13.0000,S,1,1,0,Mr,2,0,0,0
1040,1051,NaN,3,"Peacock, Mrs. Benjamin (Edith Nile)",female,26.00,0,2,SOTONOQ,13.7750,S,3,0,0,Mrs,2,0,1,0
1158,1169,NaN,2,"Faunthorpe, Mr. Harry",male,40.00,1,0,x,26.0000,S,2,0,0,Mr,3,1,0,0
582,590,0.0,3,"Murdlin, Mr. Joseph",male,28.00,0,0,A5,8.0500,S,1,1,0,Mr,2,0,0,0
950,961,NaN,1,"Fortune, Mrs. Mark (Mary McDougald)",female,60.00,1,4,x,263.0000,S,6,0,0,Mrs,4,0,0,1
836,845,0.0,3,"Culumovic, Mr. Jeso",male,17.00,0,0,x,8.6625,S,1,1,0,Mr,2,0,0,0


-----------------------------------------------------

### Converting

In [40]:
label_encoding(all_df)

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,embarked,familysize,isalone,no_fare,title,fare_category,small_f,med_f,large_f
0,1,0.0,3,155,1,22.0,1,0,A5,7.2500,2,2,0,0,3,1,1,0,0
1,2,1.0,1,286,0,38.0,1,0,PC,71.2833,0,2,0,0,4,4,1,0,0
2,3,1.0,3,520,0,26.0,0,0,STONO2,7.9250,2,1,1,0,2,2,0,0,0
3,4,1.0,1,419,0,35.0,1,0,x,53.1000,2,2,0,0,4,4,1,0,0
4,5,0.0,3,22,1,35.0,0,0,x,8.0500,2,1,1,0,3,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1294,1305,NaN,3,1116,1,28.0,0,0,A5,8.0500,2,1,1,0,3,2,0,0,0
1295,1306,NaN,1,888,0,39.0,0,0,PC,108.9000,0,1,1,0,1,4,0,0,0
1296,1307,NaN,3,1043,1,38.5,0,0,SOTONOQ,7.2500,2,1,1,0,3,1,0,0,0
1297,1308,NaN,3,1217,1,28.0,0,0,x,8.0500,2,1,1,0,3,2,0,0,0


### Some info before jumping into model 

In [41]:
missing_ratio(all_df)

,"Missing value, %",N unique value,dtype
passengerid,0.0,1299,int64
survived,32.0,2,float64
pclass,0.0,3,int64
name,0.0,1297,int64
sex,0.0,2,int64
age,0.0,98,float64
sibsp,0.0,7,int64
parch,0.0,8,int64
ticket,0.0,37,object
fare,0.0,281,float64


In [42]:
all_df = all_df.drop(['ticket'], axis = 1).reset_index(drop=True)

In [43]:
all_df.info()
all_df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299 entries, 0 to 1298
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   passengerid    1299 non-null   int64  
 1   survived       881 non-null    float64
 2   pclass         1299 non-null   int64  
 3   name           1299 non-null   int64  
 4   sex            1299 non-null   int64  
 5   age            1299 non-null   float64
 6   sibsp          1299 non-null   int64  
 7   parch          1299 non-null   int64  
 8   fare           1299 non-null   float64
 9   embarked       1299 non-null   int64  
 10  familysize     1299 non-null   int64  
 11  isalone        1299 non-null   int64  
 12  no_fare        1299 non-null   int64  
 13  title          1299 non-null   int64  
 14  fare_category  1299 non-null   int64  
 15  small_f        1299 non-null   int64  
 16  med_f          1299 non-null   int64  
 17  large_f        1299 non-null   int64  
dtypes: float

,passengerid,survived,pclass,name,sex,age,sibsp,parch,fare,embarked,familysize,isalone,no_fare,title,fare_category,small_f,med_f,large_f
671,679,0.0,3,459,0,43.0,1,6,46.9000,2,8,0,0,4,4,0,0,1
548,556,0.0,1,1273,1,62.0,0,0,26.5500,2,1,1,0,3,3,0,0,0
1111,1122,NaN,2,1153,1,14.0,0,0,65.0000,2,1,1,0,3,4,0,0,0
552,560,1.0,3,1288,0,36.0,1,0,17.4000,2,2,0,0,4,3,1,0,0
74,76,0.0,3,806,1,25.0,0,0,7.6500,2,1,1,0,3,1,0,0,0
55,57,1.0,2,1028,0,21.0,0,0,10.5000,2,1,1,0,2,2,0,0,0
1234,1245,NaN,2,528,1,49.0,1,2,65.0000,2,4,0,0,3,4,0,1,0
918,929,NaN,3,178,0,21.0,0,0,8.6625,2,1,1,0,2,2,0,0,0
1278,1289,NaN,1,414,0,48.0,1,1,79.2000,0,3,0,0,4,4,0,1,0
1099,1110,NaN,1,1252,0,50.0,1,1,211.5000,0,3,0,0,4,4,0,1,0


In [44]:
print(all_df.dtypes)

passengerid        int64
survived         float64
pclass             int64
name               int64
sex                int64
age              float64
sibsp              int64
parch              int64
fare             float64
embarked           int64
familysize         int64
isalone            int64
no_fare            int64
title              int64
fare_category      int64
small_f            int64
med_f              int64
large_f            int64
dtype: object


In [45]:
all_df.shape

(1299, 18)

In [46]:
all_df.to_csv("../data/all_df.csv", index=False)